In [8]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
import torch.optim as optim
from brevitas.nn import QuantLinear, QuantReLU
from brevitas.quant import Int8WeightPerTensorFloat, Int8ActPerTensorFloat
import onnx

class QuantizedDNN(nn.Module):
    def __init__(self):
        super(QuantizedDNN, self).__init__()
        self.fc1 = QuantLinear(10, 50, weight_bit_width=8, weight_quant=Int8WeightPerTensorFloat, bias=True)
        self.relu1 = QuantReLU(bit_width=8, act_quant=Int8ActPerTensorFloat)
        self.fc2 = QuantLinear(50, 20, weight_bit_width=8, weight_quant=Int8WeightPerTensorFloat, bias=True)
        self.relu2 = QuantReLU(bit_width=8, act_quant=Int8ActPerTensorFloat)
        self.fc3 = QuantLinear(20, 1, weight_bit_width=8, weight_quant=Int8WeightPerTensorFloat, bias=True)
        
    def forward(self, x):
        x = self.relu1(self.fc1(x))
        x = self.relu2(self.fc2(x))
        x = self.fc3(x)
        return x

# Create an instance of the network
model = QuantizedDNN()

# Generate random dataset
X = torch.randn(100, 10)  # 100 samples, 10 features each
y = torch.randn(100, 1)   # 100 target values

# Create DataLoader
dataset = TensorDataset(X, y)
dataloader = DataLoader(dataset, batch_size=10, shuffle=True)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10
for epoch in range(num_epochs):
    for inputs, targets in dataloader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}")

print("Training completed!")

# Create a dummy input tensor with the appropriate input size (batch size, input features)
dummy_input = torch.randn(1, 10)

# Define the path where the ONNX model will be saved
onnx_path = "simple_dnn.onnx"

# Export the model
torch.onnx.export(
    model,                      # Model to be exported
    dummy_input,                # Dummy input tensor
    onnx_path,                  # Output ONNX file path
    input_names=["input"],      # Name of the input layer
    output_names=["output"],    # Name of the output layer
    dynamic_axes={"input": {0: "batch_size"}, "output": {0: "batch_size"}}  # Support dynamic batch size
)

print(f"Model has been exported to {onnx_path}")

# Load the ONNX model
onnx_model = onnx.load(onnx_path)

# Check that the model is well-formed
onnx.checker.check_model(onnx_model)

print("The ONNX model is successfully exported and verified!")


Epoch [1/10], Loss: 0.9695
Epoch [2/10], Loss: 0.8194
Epoch [3/10], Loss: 0.5266
Epoch [4/10], Loss: 0.8191
Epoch [5/10], Loss: 1.3408
Epoch [6/10], Loss: 1.3926
Epoch [7/10], Loss: 0.7142
Epoch [8/10], Loss: 0.7524
Epoch [9/10], Loss: 0.8996
Epoch [10/10], Loss: 0.5814
Training completed!
Model has been exported to simple_dnn.onnx
The ONNX model is successfully exported and verified!


c:\Users\abood\AppData\Local\Programs\Python\Python312\Lib\site-packages\brevitas\quant_tensor\__init__.py:68: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  training = torch.tensor(training, dtype=torch.bool)
c:\Users\abood\AppData\Local\Programs\Python\Python312\Lib\site-packages\brevitas\quant_tensor\__init__.py:66: TracerWarning: torch.tensor results are registered as constants in the trace. You can safely ignore this warning if you use this function to create tensors out of constant variables that would be the same every time you call this function. In any other case, this might cause the trace to be incorrect.
  signed = torch.tensor(signed, dtype=torch.bool)
